In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, GlobalMaxPooling1D, BatchNormalization,Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import re
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("IMDB Dataset.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

In [ ]:
def data_processing(text):
    text = text.lower()
    #text = TextBlob(text).correct().string
    text = re.sub(r"https?://\S+", '',text, flags=re.MULTILINE)#remove urls
    text = re.sub(r'\@w+|\#','',text)#remove hashtags and mentions
    text = re.sub(r'[^\w\s]','',text)#remove panctuations
    text_tokens = word_tokenize(text)#split the sentence
    filtered_text = [w for w in text_tokens if not w in stop_words]#remove stopwords
    return " ".join(filtered_text)

In [ ]:
df['review'] = df['review'].apply(data_processing)

In [ ]:
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [ ]:
df['review'] = df['review'].apply(stemming)

In [ ]:
df.head()

In [ ]:
pos_tweets = df[df.sentiment == "positive"]
pos_tweets.head()

In [ ]:
neg_tweets = df[df.sentiment == "negative"]
neg_tweets.head()

,review,sentiment
3,basically theres family little boy jake thinks...,negative
7,show amazing fresh innovative idea 70s first a...,negative
8,encouraged positive comments film looking forw...,negative
10,phil alien one quirky films humour based aroun...,negative
11,saw movie 12 came recall scariest scene big bi...,negative


In [ ]:
mapping = {"positive" : 1, "negative" : 0}
df['sentiment'] = df['sentiment'].replace(mapping)

In [ ]:
df['sentiment'].value_counts()

In [ ]:
vect = CountVectorizer(ngram_range=(1,2)).fit(df['review'])
feature_names = vect.get_feature_names_out()

In [ ]:
X = df['review']
X = vect.transform(X)
Y = df['sentiment']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
X = df['review']
Y = df['sentiment']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
max_len = max([len(x) for x in X])
print(max_len)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

In [ ]:
#Tokenize the sentences
tokenizer = Tokenizer()
#preparing vocabulary
tokenizer.fit_on_texts(x_train)
#converting text into integer sequences
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
#padding to prepare sequences of same length
x_train = pad_sequences(x_train,maxlen=max_len)
x_test = pad_sequences(x_test,maxlen=max_len)


In [ ]:
size_of_vocabulary = len(tokenizer.word_index)+1
print(size_of_vocabulary)

In [ ]:
model = Sequential()
#embedding layer
model.add(Embedding(size_of_vocabulary,128,input_length=max_len))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
#Add loss function, metrics, optimizer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#Adding callbacks
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)
#summary
model.summary()

In [ ]:
history = model.fit(x_train, y_train,batch_size=64,epochs=5,
                    validation_data=(x_test, y_test),verbose=1,callbacks=[es,mc])

In [ ]:
loss_train = history.history['loss']
val=history.history['val_loss']
plt.plot(epochs,val,'g',label="Validation Loss")
epochs = range(1,5)
plt.plot(epochs, loss_train, 'b', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:

plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')